In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Digits recognition submission

In [ ]:
import matplotlib.pyplot as plt

from numpy.random import multivariate_normal
from scipy.linalg.special_matrices import toeplitz
from numpy.random import randn
from numpy.linalg import norm
from scipy.optimize import check_grad
import seaborn as sns
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import activations

In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')
sample_submission_df = pd.read_csv('../input/digit-recognizer/sample_submission.csv')

In [ ]:
test.head()

In [ ]:
target_col = ['label']
feature_cols = range(1, train.shape[1])
train.iloc[:, feature_cols]

In [ ]:
num_classes = 10
# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
x_train_df, x_test_df, y_train_df, y_test_df = train_test_split(train.iloc[:, feature_cols], train[target_col],
                                                    test_size = 0.2, random_state = 42)

x_train = x_train_df.to_numpy(copy=True)
x_test = x_test_df.to_numpy(copy=True)
y_train = y_train_df.to_numpy(copy=True)
y_test = y_test_df.to_numpy(copy=True)

test = test.to_numpy(copy=True)

In [ ]:
test.shape

In [ ]:
x_train     = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test      = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')

print(f'x_train shape: {x_train.shape}')
print(f'x_test shape: {x_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

print(f'train samples: {x_train.shape[0]}')
print(f'test sample: {x_test.shape[0]}')

# normalize the input data
x_train = x_train/255
x_test  = x_test/255

In [ ]:
test = test.reshape(test.shape[0], img_rows, img_cols, 1)

In [ ]:
plt.figure(figsize=(8, 2))
for i in range(6):
    plt.subplot(1, 6, i+1)
    plt.imshow(x_train[i].reshape(28, 28), cmap="gray_r")
    plt.title(f'Label = {y_train[i]}', fontsize=14)
    plt.axis("off")
plt.tight_layout()

In [ ]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_train[:10]

In [ ]:
model_ffnn = Sequential()

model_ffnn.add(Flatten(input_shape=input_shape))

model_ffnn.add(Dense(512, activation='relu'))
model_ffnn.add(Dense(256, activation='relu'))
model_ffnn.add(Dense(128, activation='relu'))
model_ffnn.add(Dense(64, activation='relu'))
model_ffnn.add(Dense(32, activation='relu'))
model_ffnn.add(Dense(16, activation='relu'))
model_ffnn.add(Dense(num_classes, activation='softmax'))
 
model_ffnn.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adagrad(),
    metrics=['accuracy']
)
model_ffnn.summary()

In [ ]:
batch_size = 64
epochs = 25
history = model_ffnn.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model_ffnn.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')

In [ ]:
y_predict = model_ffnn.predict(test)
y_predict

In [129]:
pred_labels = np.argmax(y_predict, axis=1)
predicted_labels = pred_labels.reshape((pred_labels.shape[0], 1))
labels_df = pd.DataFrame(predicted_labels, columns = ['Label'])
labels_df.index = np.arange(1, len(labels_df)+1)
labels_df.index.names = ['ImageId']
labels_df

,Label
1,2
2,0
3,9
4,7
5,3
...,...
27996,9
27997,7
27998,3
27999,9


In [ ]:
labels_df.to_csv('./submission.csv')